In [21]:
import sys
import os
import torch
import pandas as pd
from torch import nn
project_root = os.path.dirname(os.getcwd())
if project_root not in sys.path:
    sys.path.append(project_root)


In [22]:
from src.dataset import PretrainDataset
from src.model import PTSM
from src.trainer import Trainer

In [23]:
#data = pd.read_csv("data/ETTh1.csv")
data = pd.read_csv("data/Book1.csv")
data.head()

,ticker,date,high,low,close
0,AAPL,3/29/2025,223.81,217.68,217.90
1,AAPL,3/27/2025,222.00,215.80,217.95
2,AAPL,3/26/2025,220.75,213.60,214.80
3,AAPL,3/25/2025,219.20,210.90,212.50
4,AAPL,3/24/2025,223.50,217.00,221.00


In [24]:
data['group_id'] = 'A'
#data['date'] = pd.to_datetime(data['date'])
data['date'] = pd.to_datetime(data['date'], format='%m/%d/%Y')
data.head()

,ticker,date,high,low,close,group_id
0,AAPL,2025-03-29,223.81,217.68,217.90,A
1,AAPL,2025-03-27,222.00,215.80,217.95,A
2,AAPL,2025-03-26,220.75,213.60,214.80,A
3,AAPL,2025-03-25,219.20,210.90,212.50,A
4,AAPL,2025-03-24,223.50,217.00,221.00,A


In [26]:
data['time_index'] = (data['date'] - data['date'].min()).apply(lambda x: int(x.total_seconds()/60/60))
data.head()

,ticker,date,high,low,close,group_id,time_index
0,AAPL,2025-03-29,223.81,217.68,217.90,A,240
1,AAPL,2025-03-27,222.00,215.80,217.95,A,192
2,AAPL,2025-03-26,220.75,213.60,214.80,A,168
3,AAPL,2025-03-25,219.20,210.90,212.50,A,144
4,AAPL,2025-03-24,223.50,217.00,221.00,A,120


In [27]:
trainset = PretrainDataset(
    data=data,
    group_id='group_id',
    time_col='date',
    time_index='time_index',
    target='OT',
    seq_len=30 * 3,
    min_count_per_sample=20,
    stride=1,
    freq='h'
)

KeyError: 'OT'

In [7]:
print(f"size of trainset: {len(trainset)}")

size of trainset: 17401


In [8]:
model = PTSM(
    input_len=30 * 3,
    patch_size=3,
    in_channels=1,
    embed_dim=16,
    num_heads=4,
    mlp_ratio=4,
    depth=2,
    mask_ratio=0.4,
    dropout=0.1,
)

In [9]:
print(f"model size: {model.num_parameters/1e6:.2f}M")

model size: 0.01M


In [10]:
trainer = Trainer(
    model=model,
    lr=1e-2,
    max_epochs=2,
)

In [12]:
trainer.train(
    batch_size=20,
    train_dataset=trainset,
    num_workers=4,
    save_path="./",
    save_every=1,
)

Epoch: 1, Train Loss: 0.2427, Time: 61.98s
Epoch: 2, Train Loss: 0.2307, Time: 60.04s


In [13]:
checkpoint_path = "epoch_1.pth"
checkpoint = torch.load(checkpoint_path, map_location=torch.device("cpu"))

In [14]:
print(checkpoint.keys())

odict_keys(['inp_embed.weight', 'inp_embed.bias', 'patch_embed.project_layer.weight', 'patch_embed.project_layer.bias', 'pos_embed.weight', 'encoder.encoder_layer.self_attn.in_proj_weight', 'encoder.encoder_layer.self_attn.in_proj_bias', 'encoder.encoder_layer.self_attn.out_proj.weight', 'encoder.encoder_layer.self_attn.out_proj.bias', 'encoder.encoder_layer.linear1.weight', 'encoder.encoder_layer.linear1.bias', 'encoder.encoder_layer.linear2.weight', 'encoder.encoder_layer.linear2.bias', 'encoder.encoder_layer.norm1.weight', 'encoder.encoder_layer.norm1.bias', 'encoder.encoder_layer.norm2.weight', 'encoder.encoder_layer.norm2.bias', 'encoder.encoder.layers.0.self_attn.in_proj_weight', 'encoder.encoder.layers.0.self_attn.in_proj_bias', 'encoder.encoder.layers.0.self_attn.out_proj.weight', 'encoder.encoder.layers.0.self_attn.out_proj.bias', 'encoder.encoder.layers.0.linear1.weight', 'encoder.encoder.layers.0.linear1.bias', 'encoder.encoder.layers.0.linear2.weight', 'encoder.encoder.laye